## LLM Function Calling with Apigee

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/apigee-samples/blob/main/llm-function-calling/llm_function_calling.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/icon32.png?raw=true" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fapigee-samples%2Fmain%2Fllm-function-calling%2Fllm_function_calling.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/apigee-samples/main/llm-function-calling/llm_function_calling.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-function-calling/llm_function_calling.ipynb">
          <img src="https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/images/github-mark.png?raw=true" width="30" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

This is a sample on how to deploy a sample Apigee proxy and configure it as a function call.

![architecture](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-function-calling/images/arch.png?raw=1)

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.svg)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/GoogleCloudPlatform/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-function-calling/docs/cloudshell-tutorial.md)

### Install Dependencies

In [ ]:
!pip install -Uq google-genai

### Authenticate your notebook environment (Colab only)
If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

### Initialize Variables

In [ ]:
PROJECT_ID = "" # @param {type:"string"}
APIGEE_HOST = "" # @param {type:"string"}
APIKEY = "" # @param {type:"string"}
MODEL = "gemini-2.0-flash"

### Using Automatic Function Calling to Products API deployed in Apigee

In this section, you'll define a function that takes multiple parameters as inputs. Then you'll use automatic function calling in the Gemini API to make a live API call to fetch your product catalog.

![flow](https://github.com/GoogleCloudPlatform/apigee-samples/blob/main/llm-function-calling/images/flow.png?raw=1)


In [ ]:
import requests

def get_products() -> list[dict]:
    """
    Get Products

    Returns:
        list[dict]: A list of products
    """
    base_url = "https://"+APIGEE_HOST+"/v1/samples/llm-function-calling/products"

    try:
        response = requests.get(base_url, headers={"x-apikey": APIKEY})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

def get_product(
    id: int | None = None,
) -> list[dict]:
    """
    Get Products

    Returns:
        list[dict]: A list of products
    """
    base_url = "https://"+APIGEE_HOST+"/v1/samples/llm-function-calling/products/"+str(id)

    try:
        response = requests.get(base_url, headers={"x-apikey": APIKEY})
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching location data: {e}")
        return []

In [ ]:
print(get_products())
print(get_product(1))

### Test

Let's ask the Gemini model to fetch all the products from the Products API

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, Part, Tool

prompt = "List all the products in your catalog and their price"
client = genai.Client(vertexai=True, project=PROJECT_ID, location="us-central1")

response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_products], temperature=0),
)
print(response.text)

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig

prompt = "Fetch all products and find the capacity of the glass jar?"
response = client.models.generate_content(
    model=MODEL,
    contents=prompt,
    config=GenerateContentConfig(tools=[get_products, get_product], temperature=0),
)
print(response.text)